In [ ]:
# 필요한 패키지
import requests
# 데이터 처리
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# 기타 필요 : 정규식, 시간
import re
import time

!pip install tqdm
import time
from tqdm import tqdm #진행상황 확인 위한 패키지 설치

In [ ]:
f = pd.read_csv("./f.csv")
f

In [ ]:
goods_num = []
for i in f.index:
    goods_num.append(re.findall(r"\d+",f.at[i, "href"])[0])

In [ ]:
review = pd.DataFrame()
tmp = pd.DataFrame(
    data ={
        "goodsnum" : "",
        "review" : "",
        "starr" : ""
    }, index = [0]
)

In [ ]:
for goods in goods_num:
    page = 1
    while True:
        try:
            url = f"https://goods.musinsa.com/api/goods/v2/review/goods/list?similarNo=0&sort=new&selectedSimilarNo=0&page={page}&goodsNo={goods}&_=1668583739373"
            res = requests.get(url,
                            headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"})
            
            page += 1
            
            soup = BeautifulSoup(res.text, "html.parser")
            
            y = soup.find_all("div", {"class" : "review-contents__text"})
            a = soup.find_all("span", {"class" : "review-list__rating__active"})
            if y == []:
                break
            for r in range(len(y)):
                tmp.at[0, "goodsnum"] = goods
                tmp.at[0, "review"] = y[r].text
                tmp.at[0, "starr"] = re.findall(r"\d+", a[r]["style"])[0]
                review = pd.concat([review, tmp], ignore_index = True)
                for reset in tmp.columns:
                    tmp.at[0, reset] = ""
            
            if page == 11:
                break
        except :
            break